In [ ]:
本节内容：
将所需要的数据从仿真数据中提取出来构成样本集。

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
#准备文件夹保存数据
local_path=os.getcwd()
if not os.path.exists(local_path+'\\blokagedata'):
    os.mkdir(local_path+'\\blokagedata')
print(local_path+'\\blokagedata')
#指定Block数据文件夹
file_path='C:\\Users\\XDHHome\\Desktop\\Block'

# 整理样本标签

In [ ]:
# 获取SamplePipeBlock中的管道编号
for i in tqdm(range(24)):
    Pipe_code=pd.read_csv(file_path+'\\net3-'+str(i)+'\\SamplePipeBlock.csv',header=0)
    Pipe_code=Pipe_code.loc[:,'Code'].astype(int)-505000000
    Pipe_code.to_csv(local_path+'\\blokagedata\\Pipe_code_'+str(i)+'.csv',encoding='ascii')
#获取SamplePipeBlock中的管道管径
for i in tqdm(range(24)):
    Pipe_dn=pd.read_csv(file_path+'\\net3-0\\SamplePipeBlock.csv',header=0,usecols=[3])
    Pipe_dn.to_csv(local_path+'\\blokagedata\\Pipe_dn_'+str(i)+'.csv',encoding='ascii')
#获取时间节点
for i in tqdm(range(24)):
    Pipe_time=pd.DataFrame(np.ones([1160,1])*i).astype(int)
    Pipe_time.to_csv(local_path+'\\blokagedata\\Pipe_time_'+str(i)+'.csv',encoding='ascii')
#去掉没有模拟结果的样本
for i in tqdm(range(24)):
    num=[ ]
    for j in range(1,1161):
        if not os.path.exists(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Pipe.csv'):
            num.append(j-1)
    for name in ['code','dn','time']:
        data=pd.read_csv(local_path+'\\blokagedata\\Pipe_'+name+'_'+str(i)+'.csv',header=0,usecols=[1])
        data=data.drop(index=num)
        data.to_csv(local_path+'\\blokagedata\\Pipe_'+name+'_clear_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#合并24组数据
for name in ['code','dn','time']:
    data=pd.read_csv(local_path+'\\blokagedata\\Pipe_'+name+'_clear_0.csv',header=0)
    for i in tqdm(range(1,24)):
        data1=pd.read_csv(local_path+'\\blokagedata\\Pipe_'+name+'_clear_'+str(i)+'.csv',header=0)
        data=data.append(data1)
    data.to_csv(local_path+'\\blokagedata\\Pipe_'+name+'_clear_'+'24.csv',encoding='ascii',index=False)

In [ ]:
#把管道编号重编为连续整数
data=pd.read_csv(local_path+'\\blokagedata\\Pipe_code_clear_24.csv',header=0)
y=data.astype(int)
table1=np.sort(np.unique(y))
for i in tqdm(range(len(table1))):
    for j in range(len(y)):
        if y.iloc[j].item()==table1[i].item():
            y.iloc[j]=i
y.to_csv(local_path+'\\blokagedata\\Pipe_code_clear_24_sorted.csv',encoding='ascii',index=False)

In [ ]:
#合并三组标签
a=pd.read_csv(local_path+'\\blokagedata\\Pipe_code_clear_24_sorted.csv',header=0)
b=pd.read_csv(local_path+'\\blokagedata\\Pipe_dn_clear_24.csv',header=0)
c=pd.read_csv(local_path+'\\blokagedata\\Pipe_time_clear_24.csv',header=0)
abc=pd.concat([a,b,c],axis=1)
abc.columns=['code','dn','time']
abc.to_csv(local_path+'\\blokagedata\\Sample_label.csv',encoding='ascii',index=False)

# 整理样本数据

In [ ]:
#将每个时间点的Sink整理成样本
for i in tqdm(range(24)):
    sample=pd.DataFrame()
    for j in range(1,1161):
        if os.path.exists(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv'):
            sample2=pd.read_csv(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv',header=0,usecols=[2])
            sample2=sample2.drop(index=[0,1,2]).T
            sample=pd.concat([sample,sample2],axis=0)
    sample.to_csv(local_path+'\\blokagedata\\Sample_Sink_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#将每个时间点的Tank整理成样本
for i in tqdm(range(24)):
    sample=pd.DataFrame()
    for j in range(1,1161):
        if os.path.exists(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv'):
            sample2=pd.read_csv(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv',header=0,usecols=[4],nrows=3)
            sample2=sample2.T
            sample=pd.concat([sample,sample2],axis=0)
    sample.to_csv(local_path+'\\blokagedata\\Sample_Tank_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#将每个时间点的Source整理成样本
for i in tqdm(range(24)):
    sample=pd.DataFrame()
    for j in range(1,1161):
        if os.path.exists(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sources.csv'):
            sample2=pd.read_csv(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sources.csv',header=0,usecols=[4])
            sample2=sample2.T
            sample=pd.concat([sample,sample2],axis=0)
    sample.to_csv(local_path+'\\blokagedata\\Sample_Source_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#合并数据
for i in tqdm(range(24)):
    Sinksample=pd.read_csv(local_path+'\\blokagedata\\Sample_Sink_'+str(i)+'.csv',header=0)
    Tanksample=pd.read_csv(local_path+'\\blokagedata\\Sample_Tank_'+str(i)+'.csv',header=0)
    Sourcesample=pd.read_csv(local_path+'\\blokagedata\\Sample_Source_'+str(i)+'.csv',header=0)
    abc=pd.concat([Sinksample,Tanksample,Sourcesample],axis=1)
    abc.columns=range(0,96)
    abc.to_csv(local_path+'\\blokagedata\\Sample_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#合并24组数据
sample=pd.DataFrame()
for i in tqdm(range(0,24)):
    sample2=pd.read_csv(local_path+'\\blokagedata\\Sample_'+str(i)+'.csv',header=0)
    sample=pd.concat([sample,sample2],axis=0)
sample.to_csv(local_path+'\\blokagedata\\Sample_24.csv',encoding='ascii',index=False)

In [ ]:
#合并样本和标签
a=pd.read_csv(local_path+'\\blokagedata\\Sample_24.csv',header=0)
b=pd.read_csv(local_path+'\\blokagedata\\Sample_label.csv',header=0)
ab=pd.concat([a,b],axis=1)
ab.to_csv(local_path+'\\blokagedata\\Block_Sample.csv',encoding='ascii',index=False)

# 整理正常数据

In [ ]:
for i in tqdm(range(24)):
    #将每个时间点的Sink整理成样本
    sample=pd.read_csv(file_path+'\\net3-'+str(i)+'\\Result.0.Sinks.csv',header=0,usecols=[2])
    sample=sample.drop(index=[0,1,2]).T
    sample.to_csv(local_path+'\\blokagedata\\Normal_Sample_Sink_'+str(i)+'.csv',encoding='ascii',index=False)
    #将每个时间点的Tank整理成样本
    sample=pd.read_csv(file_path+'\\net3-'+str(i)+'\\Result.0.Sinks.csv',header=0,usecols=[4],nrows=3)
    sample=sample.T
    sample.to_csv(local_path+'\\blokagedata\\Normal_Sample_Tank_'+str(i)+'.csv',encoding='ascii',index=False)
    #将每个时间点的Source整理成样本
    sample=pd.read_csv(file_path+'\\net3-'+str(i)+'\\Result.0.Sources.csv',header=0,usecols=[4])
    sample=sample.T
    sample.to_csv(local_path+'\\blokagedata\\Normal_Sample_Source_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#合并数据
for i in tqdm(range(24)):
    Sinksample=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_Sink_'+str(i)+'.csv',header=0)
    Tanksample=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_Tank_'+str(i)+'.csv',header=0)
    Sourcesample=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_Source_'+str(i)+'.csv',header=0)
    abcd=pd.concat([Sinksample,Tanksample,Sourcesample],axis=1)
    abcd.columns=range(0,96)
    abcd.to_csv(local_path+'\\blokagedata\\Normal_Sample_'+str(i)+'.csv',encoding='ascii',index=False)

In [ ]:
#合并24组数据
sample=pd.DataFrame()
for i in tqdm(range(1,24)):
    sample2=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_'+str(i)+'.csv',header=0)
    sample=pd.concat([sample,sample2],axis=0)
sample.to_csv(local_path+'\\blokagedata\\Normal_Sample_24.csv',encoding='ascii',index=False)

In [ ]:
#生成标签
a=pd.DataFrame(np.ones([24,1])*116).astype(int)
b=pd.DataFrame(np.ones([24,1])*0).astype(int)#由于0%管径无模拟数据，所以用来指代正常数据
c=pd.DataFrame(range(0,24))
abc=pd.concat([a,b,c],axis=1)
abc.columns=['code','dn','time']
abc.to_csv(local_path+'\\blokagedata\\Normal_Sample_label.csv',encoding='ascii',index=False)

In [ ]:
#合并样本和标签
a=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_24.csv',header=0)
b=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_label.csv',header=0)
ab=pd.concat([a,b],axis=1)
ab.to_csv(local_path+'\\blokagedata\\Normal_Sample.csv',encoding='ascii',index=False)

In [ ]:
#过采样
a=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample.csv',header=0)
b=a
for i in range(8):
    a=a.append(b)
a.to_csv(local_path+'\\blokagedata\\Normal_Sample_oversample.csv',encoding='ascii',index=False)

In [ ]:
#合并三组标签
a=pd.read_csv(local_path+'\\blokagedata\\Block_Sample.csv',header=0)
b=pd.read_csv(local_path+'\\blokagedata\\Normal_Sample_oversample.csv',header=0)
ab=pd.concat([a,b],axis=0)
ab.to_csv(local_path+'\\blokagedata\\Sample.csv',encoding='ascii',index=False)

# 合并版

In [ ]:
col_l=pd.DataFrame()#label
col_b=pd.DataFrame()#block
col_n=pd.DataFrame()#normal
for i in tqdm(range(24)):
    # 获取SamplePipeBlock中的管道编号
    Pipe_code=pd.read_csv(file_path+'\\net3-'+str(i)+'\\SamplePipeBlock.csv',header=0)
    Pipe_code=Pipe_code.loc[:,'Code'].astype(int)-505000000
    #获取SamplePipeBlock中的管道管径比例
    Pipe_dn=pd.read_csv(file_path+'\\net3-0\\SamplePipeBlock.csv',header=0,usecols=[3])#事先在该文件里生成了一列数据
    #获取时间节点
    Pipe_time=pd.DataFrame(np.ones([1160,1])*i).astype(int)
    #去掉没有模拟结果的样本
    num=[ ]
    for j in range(1,1161):
        if not os.path.exists(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Pipe.csv'):
            num.append(j-1)
    Pipe_code=Pipe_code.drop(index=num)
    Pipe_dn=Pipe_dn.drop(index=num)
    Pipe_time=Pipe_time.drop(index=num)
    cat=pd.concat([Pipe_code,Pipe_dn,Pipe_time],axis=1)
    col_l=pd.concat([col_l,cat],axis=0)
    #获取堵塞数据
    for j in range(1,1161):
        if os.path.exists(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv'):
            sink=pd.read_csv(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv',header=0,usecols=[2]).drop(index=[0,1,2]).T.reset_index(drop=True)
            tank=pd.read_csv(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sinks.csv',header=0,usecols=[4],nrows=3).T.reset_index(drop=True)
            source=pd.read_csv(file_path+'\\net3-'+str(i)+'\\RsBlock\\Sample-Pipe-'+str(j)+'.Sources.csv',header=0,usecols=[4]).T.reset_index(drop=True)
            cat=pd.concat([sink,tank,source],axis=1)
            col_b=pd.concat([col_b,cat],axis=0)
    #获取正常数据
    sink=pd.read_csv(file_path+'\\net3-'+str(i)+'\\Result.0.Sinks.csv',header=0,usecols=[2]).drop(index=[0,1,2]).T.reset_index(drop=True)
    tank=pd.read_csv(file_path+'\\net3-'+str(i)+'\\Result.0.Sinks.csv',header=0,usecols=[4],nrows=3).T.reset_index(drop=True)
    source=pd.read_csv(file_path+'\\net3-'+str(i)+'\\Result.0.Sources.csv',header=0,usecols=[4]).T.reset_index(drop=True)
    cat=pd.concat([sink,tank,source],axis=1)
    col_n=pd.concat([col_n,cat],axis=0)
#命名列名
col_l=col_l.reset_index(drop=True) 
col_l.columns=['code','dn','time']
col_b=col_b.reset_index(drop=True)
col_b.columns=range(0,96)
col_n=col_n.reset_index(drop=True)
col_n.columns=range(0,96)
#把管道编号重编为连续整数
y=col_l.loc[:,'code'].astype(int)
table=np.sort(np.unique(y))
for i in tqdm(range(len(table))):
    for j in range(len(y)):
        if y.iloc[j].item()==table[i].item():
            y.iloc[j]=i
col_l.loc[:,'code']=y
#生成正常样本的标签
code_n=pd.DataFrame(np.ones([24,1])*116).astype(int)
dn_n=pd.DataFrame(np.ones([24,1])*0).astype(int)#由于0%管径无模拟数据，所以用来指代正常数据
time_n=pd.DataFrame(range(0,24))
cat_nl=pd.concat([code_n,dn_n,time_n],axis=1)
cat_nl.columns=['code','dn','time']
#合并样本和标签
sample_b=pd.concat([col_b,col_l],axis=1)
sample_n=pd.concat([col_n,cat_nl],axis=1)
#合并堵塞和正常数据，并过采样
for i in range(9):
    sample_b=pd.concat([sample_b,sample_n],axis=0)
sample_b=sample_b.reset_index(drop=True)
sample_b.to_csv(local_path+'\\blokagedata\\Sample.csv',encoding='ascii',index=False)

# 得到残差

In [ ]:
# 做残差
z=np.array(sample.iloc[-24::,:])
for i in tqdm(range(0,24)):
    sample.iloc[np.where(sample.loc[:,'time']==i)[0],range(0,96)]=sample.iloc[np.where(sample.loc[:,'time']==i)[0],range(0,96)]-z[i,:]
sample=sample.drop(np.where(sample.loc[:,'code']==116)[0])
sample.to_csv(local_path+'\\blokagedata\\Sample_116.csv',encoding='ascii',index=False)

In [ ]:
# 做残差，取变化百分比
sample=pd.read_csv(local_path+'\\blokagedata\\Sample_96_117.csv',header=0)
z=np.array(sample.iloc[-24::,range(0,96)])
for i in tqdm(range(0,24)):
    sample.iloc[np.where(sample.loc[:,'time']==i)[0],range(0,96)]=(sample.iloc[np.where(sample.loc[:,'time']==i)[0],range(0,96)]-z[i,:])/z[i,:]
sample2=sample.drop(np.where(sample.loc[:,'code']==116)[0])
sample.to_csv(local_path+'\\blokagedata\\Sample_116_percent.csv',encoding='ascii',index=False)